In [ ]:
# importing packages
import cv2
from skimage.measure import compare_ssim
import argparse
import imutils
import os
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
# setting file directory
os.chdir(r'C:\Users\gunal\Desktop\buffalo automation')

In [ ]:
#function to get frames from video
def getframe(video,parse):
    
    readvid = cv2.VideoCapture(video)
    
    # fps of the video
    fps = int(readvid.get(cv2.CAP_PROP_FPS))
    
    # Used as counter variable 
    count = 0

    # checks whether frames were extracted 
    success = 1

    imagelist = []

    while success: 

        # vidObj object calls read 
        # function extract frames 
        success, image = readvid.read() 

        # Saves the frames with frame-count
        if count % (fps*parse) == 0:
            imagelist.append(image)


        count += 1
    
    # final frames for analysis
    return imagelist
    

In [ ]:
# find the unique frame with the given similarity
def uniqueframes(frames,similarity):
    
    unique_frame = frames[0]
    
    gray1 = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    
    for i in range(0,len(frames)-1):
        gray2 = cv2.cvtColor(frames[i+1], cv2.COLOR_BGR2GRAY)
        
        # score gives a value in range (-1,1) with 1 being perfect match
        (score, diff) = compare_ssim(gray1, gray2, full=True)
        diff = (diff * 255).astype("uint8")
        
        if score <= similarity:
            unique_frame.append(frames[i+1])
            gray1 = cv2.cvtColor(frames[i+1], cv2.COLOR_BGR2GRAY)
    
    # return the final unique frames
    return unique_frame
            

In [ ]:
# save images in the exit path
def saveimages(path,images):
    
    for i in range(0,len(images)):
        
        cv2.imwrite( path + "\\frame%d.jpg" % i, images[i])

In [ ]:
if __name__ == "__main__":
    
    
    while True:
        
        # getting the file name
        print('Enter \'exit\' to stop program else enter file name to continue')
        path = input('file name: ')
        
        if path.lower == 'exit':
            break

        # getting frame intervals for parsing 
        video_parse = int(input('video parse time to get frames: '))

        # getting max level of similarity between frames
        similarity = int(input('% of similarity from 0 - 100%: '))

        # returns the parsed frames from the total video file
        images = getframe(path,video_parse)


        # genearting list from 0-100 for minmax scalar
        percent = np.arange(0,101,dtype='int')

        # scaling 0-100 range to match the score of image difference scale (-1,1)
        a = np.array(similarity)
        minmax = MinMaxScaler(feature_range=(-1,1))
        minmax.fit(percent.reshape(-1,1))
        similarity = minmax.transform(a.reshape(-1, 1))[0])

        # returns all the unique frames
        unique_frame = uniqueframes(frames=images, similarity=similarity)

        # count of unique franes
        print('total unique frames: ', len(unique_frame))


        # save the images
        pathout = input('exit path for saving the unique images: ')
        saveimages(pathout,unique_frame)
